In [1]:
from datasets import Dataset

ds = Dataset.from_parquet("/drive2/kaggle/pii-dd/data/new_ds2.pq")

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
ds[0]

{'id': 'e_jobz51hh',
 'essay': " Introduction - Identifying the Challenge:\n\nIn my role as a User Experience Designer at a technology startup in San Francisco, I encountered a complex challenge that required a thoughtful and innovative solution. Our team was tasked with redesigning the user experience of our mobile application to better meet the needs of a diverse user group, spanning various age ranges, cultures, ethnicities, and abilities. This challenge was significant due to the wide array of user needs and preferences to consider, as well as the potential impact on overall customer satisfaction and conversion rates.\n\nSelection of the Tool or Approach:\n\nTo address this complex challenge, I chose to apply the human-centered design (HCD) approach, a methodology that emphasizes empathy, collaboration, and iteration throughout the design process. I selected this approach due to its relevance to the challenge, as HCD focuses on understanding and meeting the needs of diverse user gr

In [12]:
import json
import argparse
from itertools import chain

from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification, pipeline
from datasets import Dataset
import numpy as np

def tokenize(example, tokenizer, max_length):
    # text = []
    # token_map = []
    
    # idx = 0
    
    # for t, ws in zip(example["tokens"], example["trailing_whitespace"]):
        
    #     text.append(t)
    #     token_map.extend([idx]*len(t))
    #     if ws:
    #         text.append(" ")
    #         token_map.append(-1)
            
    #     idx += 1
            
        
    # tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length)
    
    tokenized = tokenizer(example["essay"], return_offsets_mapping=True, max_length=max_length)

        
    return {
        **tokenized,
        # "token_map": token_map,
    }


def main(model_path, max_length):

    class A:

        model_path = ""
        max_length = ""

    args = A

    args.model_path = model_path
    args.max_length = max_length
    
    # data = json.load(open("/kaggle/input/pii-detection-removal-from-educational-data/test.json"))
    
    # ds = Dataset.from_dict({
    #     "full_text": [x["full_text"] for x in data],
    #     "document": [x["document"] for x in data],
    #     "tokens": [x["tokens"] for x in data],
    #     "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    # })

    ds = Dataset.from_parquet("/drive2/kaggle/pii-dd/data/new_ds2.pq")
    
    tokenizer = AutoTokenizer.from_pretrained(args.model_path)
    ds = ds.map(
        tokenize, 
        fn_kwargs={"tokenizer": tokenizer, "max_length": args.max_length}, 
        num_proc=2,
    )
    
    model = AutoModelForTokenClassification.from_pretrained(args.model_path)

    # collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

    # args = TrainingArguments(
    #     ".", 
    #     per_device_eval_batch_size=4, 
    #     report_to="none",
    # )
    
    # trainer = Trainer(
    #     model=model, 
    #     args=args, 
    #     data_collator=collator, 
    #     tokenizer=tokenizer,
    # )

    pl = pipeline("ner", model=model, tokenizer=tokenizer, device=0)

    preds = []
    for i in range(0, len(ds), 16):
        texts = ds[i:min(len(ds), i+16)]["essay"]
        preds.extend(pl(texts))
    
    import pickle

    with open("preds.pkl", "wb") as f:
        pickle.dump(preds, f)
    
    # predictions = trainer.predict(ds).predictions

    # ds.to_parquet("test_ds.pq")
    
    # np.save("preds.npy", predictions)

In [13]:
model_path = "/drive2/kaggle/pii-dd/models/dv3b-v1/output/checkpoint-240"

main(model_path, 2000)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/nicholas/miniconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [14]:
import pickle

with open("preds.pkl", "rb") as f:
    preds = pickle.load(f)

In [16]:
count = 0



for p in preds:

    if len(p) > 0:
        count += 1

count

133

In [24]:
len([x for x in ds["essay"] if "Word Count" in x])

22

In [25]:
[x.index("Word Count")/len(x) for x in ds["essay"] if "Word Count" in x]

[0.9577735124760077,
 0.9957480733457348,
 0.9960460100647016,
 0.9969167523124358,
 0.9973679592911037,
 0.9358752166377816,
 0.9916497404649064,
 0.9764337851929092,
 0.9966659720775162,
 0.9966624947851481,
 0.9971356963838167,
 0.9872689938398357,
 0.9791431792559189,
 0.996,
 0.9968287526427061,
 0.9680558493759255,
 0.9956803455723542,
 0.9964243146603099,
 0.9968159203980099,
 0.9970828471411902,
 0.9971687429218573,
 0.996317157712305]

In [60]:
def clean_essays(x, debug=False):

    if debug:
        import pdb; pdb.set_trace()

    if "Retrieved from" in x and x.index("Retrieved from")/len(x) > 0.8:

        x = x[:x.index("Retrieved from")]

    
    if "Word Count" in x and x.index("Word Count")/len(x) > 0.8:
        x = x[:x.index("Word Count")]


    if "References" in x and x.index("References")/len(x) > 0.7:
        x = x[:x.index("References")]

    pattern = "\n\nReference"
    if pattern in x and x.index(pattern)/len(x) > 0.7:
        x = x[:x.index(pattern)]

    if "**Note:**" in x and x.index("**Note:**")/len(x) > 0.8:
        x = x[:x.index("**Note:**")]

    if "Please note" in x and x.index("Please note")/len(x) > 0.8:
        x = x[:x.index("Please note")]

    if "\n\n" in x and x.rindex("\n\n")/len(x) > 0.95:
        x = x[:x.rindex("\n\n")]

    return x


In [59]:
t = " ' Challenge: Addr\n\nReference:essing the growing energy demands of expanding cities while minimizing carbon emissions and environmental impact.\n\nAs an environmental engineer at a leading sustainable energy company, I have confronted numerous challenges that require innovative solutions to balance economic growth with environmental stewardship. One such complex challenge is addressing the increasing energy demands of expanding cities while minimizing carbon emissions and environmental impact. This issue is significant because urban areas account for approximately 70% of global carbon dioxide emissions, and their energy consumption is projected to double by 2050 due to population growth and economic development. Consequently, there is an urgent need for cleaner and more efficient energy solutions to mitigate the environmental consequences of urban expansion.\n\nSelection of the Tool or Approach:\n\nTo tackle this challenge, I chose to apply the Integrated Energy Systems (IES) approach, which combines various energy conversion technologies, energy storage systems, and demand-side management strategies to optimize energy supply and demand. I selected this approach because of its comprehensive nature and its potential to provide a long-term solution for urban energy systems. Furthermore, IES has been successfully implemented in several demonstration projects worldwide, showcasing its applicability and effectiveness. Alternatives like relying solely on renewable energy sources or implementing energy efficiency measures were considered but discarded since they may not adequately address the complex interplay between energy supply and demand in rapidly growing urban areas.\n\nApplication of the Tool or Approach:\n\nTo apply the IES approach, I first conducted a thorough analysis of the energy supply and demand patterns in the target urban area. Based on this analysis, I identified suitable energy conversion technologies, including combined heat and power (CHP) systems, renewable energy sources, and waste-to-energy plants. Next, I incorporated energy storage systems, such as batteries and thermal storage, to balance the intermittent nature of renewable energy sources and ensure a reliable energy supply.\n\nIn addition, I integrated demand-side management strategies, such as energy conservation measures, demand response programs, and smart grid technologies, to optimize energy consumption patterns. By utilizing these components, I developed an optimized IES that balanced energy supply and demand while minimizing carbon emissions and environmental impact.\n\nThroughout the application process, I collaborated with various stakeholders, including local government officials, utility companies, and community representatives, to ensure the feasibility and acceptability of the proposed IES. Furthermore, I engaged in extensive public outreach to raise awareness of the benefits of the IES approach and garner support for its implementation.\n\nAnalysis and Insight:\n\nThe IES approach has proven to be effective in addressing the complex challenge of urban energy demands while minimizing carbon emissions and environmental impact. The optimized IES successfully balanced energy supply and demand, reducing energy losses and enhancing the overall efficiency of the urban energy system.\n\nHowever, the implementation of the IES was not without limitations. The initial capital investment required for the deployment of energy conversion and storage technologies was relatively high. Additionally, coordinating various stakeholders and ensuring the compatibility of different energy systems posed significant challenges.\n\nDespite these limitations, the IES approach has led to several unexpected outcomes and lessons learned. For instance, the public engagement process revealed a strong desire for cleaner and more reliable energy sources, which further motivated the adoption of the IES. Moreover, the implementation process fostered collaboration among stakeholders, leading to the creation of a robust network for knowledge sharing and future partnerships.\n\nConclusion - Future Applications:\n\nThe key insights gained from applying the IES approach to address urban energy demands highlight the need for long-term, comprehensive solutions that consider both energy supply and demand. Moreover, the importance of engaging various stakeholders and the public throughout the implementation process cannot be overstated.\n\nMoving forward, these insights can inform the application of similar tools in addressing other complex challenges related to energy systems, such as electrifying transportation or integrating renewable energy sources into existing power grids. In particular, the IES approach can be expanded to incorporate emerging technologies like electric vehicles, hydrogen fuel cells, and advanced energy management systems. By building upon the foundational principles of IES, we can develop innovative and sustainable solutions to confront the evolving energy landscape.\n\nIn conclusion, my experience applying the IES approach to tackle the complex challenge of urban energy demands has demonstrated both the strengths and limitations of this tool. Despite the challenges encountered, the IES approach has proven to be effective in addressing energy supply and demand while minimizing carbon emissions and environmental impact. By building on the lessons learned and incorporating emerging technologies, we can enhance the effectiveness of the IES approach and pave the way for a more sustainable future.\n\n--------------------',"

clean_essays(t, debug=True)

> /tmp/ipykernel_200747/3808013601.py(6)clean_essays()
      4         import pdb; pdb.set_trace()
      5 
----> 6     if "Retrieved from" in x and x.index("Retrieved from")/len(x) > 0.8:
      7 
      8         x = x[:x.index("Retrieved from")]

0.9956709956709957
" ' Challenge: Addressing the growing energy demands of expanding cities while minimizing carbon emissions and environmental impact.\n\nAs an environmental engineer at a leading sustainable energy company, I have confronted numerous challenges that require innovative solutions to balance economic growth with environmental stewardship. One such complex challenge is addressing the increasing energy demands of expanding cities while minimizing carbon emissions and environmental impact. This issue is significant because urban areas account for approximately 70% of global carbon dioxide emissions, and their energy consumption is projected to double by 2050 due to population growth and economic development. Consequently, there i

In [61]:
new_essays = [clean_essays(x) for x in ds["essay"]]

In [63]:
len([y for x,y in zip(ds["essay"], new_essays) if x != y])

339

In [64]:
len([y for x,y in zip(ds["essay"], new_essays) if "[URL" in y])

14

In [65]:
len([y for x,y in zip(ds["essay"], new_essays) if "[URL" in x])

38

In [68]:
len([y for x,y in zip(ds["essay"], new_essays) if "http" in y])

6

In [103]:
clean = [x for x, p in zip(new_essays, preds) if len(p) == 0]
clean = [x for x in clean if "http" not in x]
clean = [x for x in clean if "[URL" not in x]
clean = [x.strip().strip(" \n-") for x in clean]

len(clean)

2355

In [142]:
import random

random.sample(clean, 1)

["**Introduction - Identifying the Challenge**\n\nIn the dynamic world of digital marketing, one of the most complex challenges is crafting a comprehensive strategy that effectively reaches and engages a niche target audience. This is especially true for tech startups attempting to break into highly competitive markets. For the past four years, I have been working as a Digital Marketing Specialist at a tech startup, and I've faced this challenge head-on.\n\nThe complexity of this challenge stems from the increasingly fragmented digital landscape and the diverse preferences of niche audiences. Traditional marketing techniques often fail to deliver the desired results, leaving digital marketers in search of innovative strategies. The successful identification and implementation of these strategies require a deep understanding of the latest digital marketing tools and techniques, as well as the ability to adapt quickly to emerging trends and user behaviors.\n\n**Selection of the Tool or A

In [143]:
import json

json.dump(clean, open("clean_essays.json", "w"))

In [19]:
for p, x in zip(preds, ds["essay"]):
    if len(p) > 0:
        print(x)
        import pdb; pdb.set_trace()

 **INTRODUCTION - IDENTIFYING THE CHALLENGE**

The exponential growth of digital platforms and the increasing reliance on online searches for products, services, and information have made Search Engine Optimization (SEO) and Search Engine Marketing (SEM) crucial components of digital marketing strategies. As a Digital Marketer (Search Specialist), I encountered a significant challenge: how to optimize a client's online presence amid constant changes in search engine algorithms while ensuring that their unique value proposition reaches the right audience? The complexity of this challenge lies in striking a delicate balance between catering to search engine requirements and addressing user needs.

**SELECTION OF THE TOOL OR APPROACH**

To tackle this challenge, I chose to employ an Integrated Search Strategy – a approach that combines SEO, SEM, and Content Marketing tactics. I selected this approach because it allows for a comprehensive, multi-faceted optimization of a client's online pr

In [7]:
preds = np.load("preds.npy")

count = 0


for p in preds:

    break
    # if p.argmax(-1):

In [9]:
p.argmax(-1)

array([12, 12, 12, ...,  0,  0,  0])

In [10]:
import json
id2label = json.load(open("/drive2/kaggle/pii-dd/models/dv3b-v1/output/checkpoint-240/config.json"))["id2label"]

In [11]:
id2label

{'0': 'B-EMAIL',
 '1': 'B-ID_NUM',
 '2': 'B-NAME_STUDENT',
 '3': 'B-PHONE_NUM',
 '4': 'B-STREET_ADDRESS',
 '5': 'B-URL_PERSONAL',
 '6': 'B-USERNAME',
 '7': 'I-ID_NUM',
 '8': 'I-NAME_STUDENT',
 '9': 'I-PHONE_NUM',
 '10': 'I-STREET_ADDRESS',
 '11': 'I-URL_PERSONAL',
 '12': 'O'}

In [ ]:
if not TRAINING:
    
    import numpy as np
    import json
    from datasets import Dataset
    import pandas as pd
    from pathlib import Path

    config = json.load(open(Path(INFERENCE_MODEL_PATH) / "config.json"))

    id2label = config["id2label"]

    preds = np.load("preds.npy")

    ds = Dataset.from_parquet("test_ds.pq")

    preds = preds.argmax(-1)

    triplets = []
    document, token, label, token_str = [], [], [], []
    for p, token_map, offsets, tokens, doc in zip(preds, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"]):

        for token_pred, (start_idx, end_idx) in zip(p, offsets):
            label_pred = id2label[str(token_pred)]

            if start_idx + end_idx == 0: continue

            if token_map[start_idx] == -1: 
                start_idx += 1

            # ignore "\n\n"
            while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                start_idx += 1

            if start_idx >= len(token_map): break

            token_id = token_map[start_idx]

            # ignore "O" predictions and whitespace preds
            if label_pred != "O" and token_id != -1:
                triplet = (label_pred, token_id, tokens[token_id])

                if triplet not in triplets:
                    document.append(doc)
                    token.append(token_id)
                    label.append(label_pred)
                    token_str.append(tokens[token_id])
                    triplets.append(triplet)


    df = pd.DataFrame({
        "document": document,
        "token": token,
        "label": label,
        "token_str": token_str
    })

    df["row_id"] = list(range(len(df)))

    display(df.head(50))


    df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)

In [12]:
ds = Dataset.from_parquet("/drive2/kaggle/pii-dd/data/ds-v2.pq")
ds

Dataset({
    features: ['essay', 'name', 'personal_url', 'address', 'username', 'email', 'id_num', 'phone_number', 'social', 'another_social', 'another_personal', 'tokens', 'labels', 'trailing_whitespace'],
    num_rows: 2355
})

In [11]:
x = ds[0]

[(i, t,l) for i, (t,l) in enumerate(list(zip(x["tokens"], x["labels"]))) if l != "O"]

[(0, 'Tiburce', 'B-NAME_STUDENT'),
 (1, 'Evans', 'I-NAME_STUDENT'),
 (3, 'https://www.instagram.com/tiburce-evans', 'B-URL_PERSONAL'),
 (7, 'bLBeoRIe', 'B-ID_NUM'),
 (9, '001', 'B-PHONE_NUM'),
 (10, '-', 'I-PHONE_NUM'),
 (11, '691', 'I-PHONE_NUM'),
 (12, '-', 'I-PHONE_NUM'),
 (13, '518', 'I-PHONE_NUM'),
 (14, '-', 'I-PHONE_NUM'),
 (15, '9820x5621', 'I-PHONE_NUM'),
 (1095, 'Tiburce', 'B-NAME_STUDENT'),
 (1096, 'Evans', 'I-NAME_STUDENT')]

In [10]:
import re

text = " ".join(x['tokens'])


re.findall("Tiburce Evans", text)

['Tiburce Evans', 'Tiburce Evans']

In [13]:
x = [d for d in ds if d["name"] == "Mike Madison"][0]

In [15]:
[(i, t,l) for i, (t,l) in enumerate(list(zip(x["tokens"], x["labels"]))) if l != "O"]

[(0, 'Mike', 'B-NAME_STUDENT'),
 (1, 'Madison', 'I-NAME_STUDENT'),
 (4, '64886661046', 'B-ID_NUM'),
 (6, 'https://www.facebook.com/mike_m', 'B-URL_PERSONAL'),
 (627, 'Mike', 'B-NAME_STUDENT'),
 (628, 'Madison', 'I-NAME_STUDENT'),
 (631, '64886661046', 'B-ID_NUM'),
 (633, 'https://www.facebook.com/mike_m', 'B-URL_PERSONAL'),
 (912, 'Mike', 'B-NAME_STUDENT'),
 (913, 'Madison', 'I-NAME_STUDENT')]

In [17]:
text = " ".join(x['tokens'])

print(text)

re.findall("Mike Madison", text)

Mike Madison , num 64886661046 - https://www.facebook.com/mike_m 

 Introduction - Identifying the Challenge : 

 In the dynamic field of digital marketing , one of the most significant and complex challenges I have encountered is creating a cohesive and engaging multi - channel marketing strategy that effectively reaches and converts a diverse audience . This challenge is especially pertinent for startups , where resources and budgets are often limited , and the margin for error is slim . With the ever - growing number of digital platforms and channels , it can be overwhelming to identify and prioritize the most effective touchpoints for a specific target audience . Moreover , ensuring a seamless and consistent brand experience across all channels adds another layer of complexity to the process . 

 Selection of the Tool or Approach : 

 To tackle this challenge , I turned to the well - established marketing framework , the Marketing Funnel . The Marketing Funnel is a systematic appro

['Mike Madison', 'Mike Madison', 'Mike Madison']

In [18]:
import json

data = json.load(open("/drive2/kaggle/pii-dd/data/mixtral-8x7b-v1/mixtral-8x7b-v1.json"))

In [19]:
data[0].keys()

dict_keys(['document', 'full_text', 'tokens', 'labels', 'trailing_whitespace'])

In [20]:
x = [d for d in data if d["document"] == "oyxdwbhrro"][0]

In [22]:
[(i, t,l) for i, (t,l) in enumerate(list(zip(x["tokens"], x["labels"]))) if l != "O"]

[(0, 'Mike', 'B-NAME_STUDENT'),
 (1, 'Madison', 'I-NAME_STUDENT'),
 (4, '64886661046', 'B-ID_NUM'),
 (6, 'https://www.facebook.com/mike_m', 'B-URL_PERSONAL'),
 (627, 'Mike', 'B-NAME_STUDENT'),
 (628, 'Madison', 'I-NAME_STUDENT'),
 (631, '64886661046', 'B-ID_NUM'),
 (633, 'https://www.facebook.com/mike_m', 'B-URL_PERSONAL'),
 (912, 'Mike', 'B-NAME_STUDENT'),
 (913, 'Madison', 'I-NAME_STUDENT')]